# FEDAVG BENCHMARK

We try to reproduce the results of the paper [Federated Optimization in Heterogeneous Networks](https://arxiv.org/abs/1812.06127) on the MNIST dataset. The paper introduces a new algorithm called FedProx, which is a Federated Averaging algorithm with a proximal term added to the loss function to encourage the local models to be close to each other. The algorithm is tested on the MNIST dataset with a [MNIST_LR](../fluke/nets.py:549)(a simple logistic regression model). The paper shows that FedProx outperforms FedAvg in terms of convergence speed and accuracy.

## Setup of the experiment

### Loading and splitting the dataset

In [9]:
from fluke.data.datasets import Datasets
dataset = Datasets.get("mnist", path="../data")

In [2]:
from fluke.data import DataSplitter
splitter = DataSplitter(dataset=dataset,
                        distribution="iid")

### Setting up the evaluator

In [3]:
from fluke.evaluation import ClassificationEval, Evaluator
from fluke import GlobalSettings

evaluator = ClassificationEval(eval_every=1, n_classes=dataset.num_classes)
GlobalSettings().set_evaluator(evaluator)

### Setting up the hyperparameters and the model

In [4]:
from fluke import DDict

client_hp = DDict(
    batch_size=10,
    local_epochs=50,
    loss="CrossEntropyLoss",
    mu=1,
    optimizer=DDict(
      lr=0.03),
    scheduler=DDict(
      gamma=1,
      step_size=1)
)

alg_hp = DDict(
    client = client_hp,
    server=DDict(weighted=True),
    model="MNIST_LR")

In [5]:
from fluke.algorithms.fedprox import FedProx
algorithm = FedProx(1000, splitter, alg_hp)

### Setting up the logger

In [6]:
from fluke.utils.log import Log
logger = Log()
algorithm.set_callbacks(logger)

## Running the experiment

In [ ]:
algorithm.run(40, 0.01)